In [3]:
from torchvision.models import densenet161
from torch import nn
import numpy as np
from PIL import Image
from utilmaps import *
import os, sys, cv2, torch, json, shutil
from matplotlib import cm
import scipy as sp
import scipy.ndimage
from util_storage import *

    


In [4]:
dataset_folder = './dataset/'
target_fold = 0

In [5]:
def loadMask(path, size_limit=None):
    with Image.open(path) as im:
        mask = np.array(im.convert('1')).astype(bool)
        
        if __name__=='__main__':
            fig, axs = plt.subplots(1, 2)
            axs[0].imshow(mask)
            
        if size_limit is not None:
            s = generate_binary_structure(2,2)
            label_structures, num_features = label(mask, s)

            for connected_component_id in [x+1 for x in range(num_features)]:
                current_component = label_structures == connected_component_id

                if np.nansum(current_component) > size_limit:
                    mask[current_component] = False
    return mask

def generateMap(image_name, image_mask, output_name):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    jet_cmap = cm.get_cmap('jet')

    try:
        image_original =  np.array(Image.open(image_name))[:,:,0].astype(float)/255
    except:
        image_original =  np.array(Image.open(image_name)).astype(float)/255

    image_mask = np.array(Image.open(image_mask).convert('L')).astype(bool)    
    
    # FASE 1
    model = densenet161()
    model.classifier = nn.Linear(model.classifier.in_features, 2)
    model.load_state_dict(torch.load('model_64.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    builder = MapBuilder(model,64,55,model_input_size=224,batch_size=500)

    roi_mask = builder.generateMap(image_original,image_mask)*image_mask
    
    
    th = 0.66
    im = Image.fromarray((roi_mask*255).astype('uint8'))
    im.save(output_name.replace('.','_phase1.'))
    im = Image.fromarray(((roi_mask > th)*255).astype('uint8'))
    im.save(output_name.replace('.','_phase1_bin.'))
            
    roi_mask = roi_mask > th
    builder.batch_size = 4000
    del model
    torch.cuda.empty_cache()
    
    # FASE 2
    model = densenet161()
    model.classifier = nn.Linear(model.classifier.in_features, 2)
    model.load_state_dict(torch.load('model_30.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    
    builder = MapBuilder(model,30,25,model_input_size=64)
    microcyst_map = np.nan_to_num(builder.generateMap(image_original,roi_mask))
    
    microcyst_map = sp.ndimage.filters.gaussian_filter(microcyst_map, 3, mode='constant')
    
    
    rgbimage = cv2.cvtColor((image_original*255).astype('uint8'),cv2.COLOR_GRAY2RGB)
    generated_map = cv2.addWeighted((jet_cmap(microcyst_map)[:,:,0:3]*255).astype('uint8'),0.5,rgbimage,0.5,0)  * np.dstack((image_mask,image_mask,image_mask))

    im = Image.fromarray(generated_map)
    im.save(output_name)
    
    
    return microcyst_map

In [6]:
images_path = dataset_folder + 'ORIGINALS/'
masks_path = dataset_folder + 'MASKS/'
microcysts_path = dataset_folder + 'MICROCYSTS/'

images_names = list(os.listdir(images_path))
masks_names = list(os.listdir(masks_path))
microcysts_names = list(os.listdir(microcysts_path))


fold_data = loadObj('FOLD_DATA_FN_L_6V2T1')[target_fold]
fold_keys = list(fold_data.keys())[::-1]


for key in ['test']:
    try:
        shutil.rmtree(key)
    except:
        None
    os.makedirs(key,exist_ok = True)


In [ ]:
for fold_stage in ['test']:
    for image_name in fold_data[fold_stage]:
        image_name = image_name.replace('dataset/','').replace('.npy','').replace('.png','').replace('png','.png').replace('tiff','.tiff').replace('tif','.tif').replace('bmp','.bmp').replace('jpg','.jpg').replace('jpeg','.jpeg')
        image_path = images_path + image_name
        mask_path = masks_path + image_name.replace('.','')+'.png'
        microcyst_path = microcysts_path + image_name.replace('.','')+'.png'
        generated_map = generateMap(image_path, mask_path, '{}/'.format(fold_stage) + image_name.replace('.','') + '.png')
        np.save('{}/'.format(fold_stage) + image_name + '.npy',generated_map)